In [9]:
import importlib
from __imports__ import *
import data, LSTM_returns, optimization

SOL_df = data.load_asset('SOLUSDT', sampling='6h')
SOL_df = data.subset(SOL_df, start=pd.Timestamp('2021-01-01 08:00:00'), end=pd.Timestamp('2026-01-01'))
data.report_and_print_gaps(SOL_df, delta=pd.Timedelta('6h'))

SOL_df = data.add_returns(SOL_df)
SOL_df = data.add_fear_and_greed(SOL_df, includeCategory=False)
SOL_df.dropna(inplace=True)
SOL_df = data.train_test_split(SOL_df)
print(SOL_df.columns)


Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Quote asset volume',
       'Number of trades', 'Taker buy base asset volume',
       'Taker buy quote asset volume', 'Return', 'Return_Target', 'F&G',
       'SPLIT'],
      dtype='object')


In [10]:
# vs. normalize all features AND targets
DF = SOL_df.copy()

numerical_columns = categorical_columns = "all"
exclude_columns = []
DF_features = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume',
       'F&G', 'Return']
DF_features_simple = ['Open', 'High', 'Low', 'Close', 'Volume', 'Return']
DF_features_nosentiment = ['Open', 'High', 'Low', 'Close', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume',
       'Return']
DF, DF_columns = data.normalize_data(DF, exclude_columns=exclude_columns)

DF_columns

{'Open': MinMaxScaler(),
 'High': MinMaxScaler(),
 'Low': MinMaxScaler(),
 'Close': MinMaxScaler(),
 'Volume': MinMaxScaler(),
 'Quote asset volume': MinMaxScaler(),
 'Number of trades': MinMaxScaler(),
 'Taker buy base asset volume': MinMaxScaler(),
 'Taker buy quote asset volume': MinMaxScaler(),
 'Return': MinMaxScaler(),
 'Return_Target': MinMaxScaler(),
 'F&G': MinMaxScaler(),
 'SPLIT': 'standard'}

In [ ]:
import json
import optuna
from IPython.display import display, HTML
import joblib

importlib.reload(LSTM_returns)

Log = []

def objective(trial):
    entry = {}
    # Example of hyperparameter suggestions
    hyperparams = {
        'lstm_units': trial.suggest_int('lstm_units', 50, 90),
        'num_layers': trial.suggest_int('num_layers', 3, 5),
        'epochs': 20,
        'batch_size': trial.suggest_categorical('batch_size', [32, 64, 128]),
        'dropout': trial.suggest_int('dropout_rate', 0, 2) / 20,
        'learning_rate': trial.suggest_categorical('learning_rate', [0.00001, 0.0001, 0.001, 0.01, 0.1]),
        'LSTM_activation': trial.suggest_categorical('lstm_activation', ['relu', 'tanh']),
        '__+ABD_exceptLast': False,
        '__finalAct': 'sigmoid',
        'optimizer': 'adam',
        'features': trial.suggest_categorical('features', ['all', 'simple', 'nosentiment']),
        # 'loss': trial.suggest_categorical('loss', ['mean_squared_error', 'mean_absolute_error', 'huber']),
    }
    episode_lengths = trial.suggest_int('episode_length', 10, 60)
    hyperparams['episode_length'] = episode_lengths
    hyperparams['layers'] = [hyperparams['lstm_units']] * hyperparams['num_layers']
    hyperparams['loss'] = 'mean_squared_error'
    # hyperparams['batch_size'] = 128
    # hyperparams['epochs'] = 1  # Set to 1 for quick testing; increase for actual training

    
    # Placeholder for model training and evaluation
    features = {
        'all': DF_features,
        'simple': DF_features_simple,
        'nosentiment': DF_features_nosentiment
    }[hyperparams['features']]

    train_X_y = LSTM_returns.vectorize_train_data(DF, features=features, episode_length=episode_lengths)
    hyperparams['input_shape'] = train_X_y['train_X'].shape[1:]
    
    LSTM = LSTM_returns.create(hyperparams)
    results = LSTM_returns.train(LSTM, hyperparams, train_X_y, evaluateAtEnd=False)

    entry['hyperparams'] = hyperparams
    entry['results'] = results
    entry['plot-title'] = f"EL: {episode_lengths}; HP: {json.dumps(hyperparams)}"
    entry['model'] = LSTM
    entry['train_X_y'] = train_X_y
    Log.append(entry)

    return results.history['loss'][-1]  # Return the last loss value as the score


# Example Optuna study
study = optuna.create_study(direction='minimize')  # or 'maximize' depending on the objective

for i in range(50):
    study.optimize(objective, n_trials=1)

    # best_entry = [
    #     entry for entry in Log if entry['results'].history['loss'][-1] == study.best_value
    # ][-1]

    latest_entry = Log[-1]
    Log = []

    # Print the best hyperparameters
    # Display the best hyperparameters and score in purple
    display(HTML(f"""
    <div style="color: purple; background: orange; font-weight: bold;">
        Trial {i*10+1} completed.<br>
        Best hyperparameters: {study.best_params}<br>
        Best score (MSE): {study.best_value}<br>
    </div>
    """))

    # Display the best entry plot
    plt = LSTM_returns.all_folds_plot(
        latest_entry['model'], latest_entry['train_X_y']
    )
    plt.title(latest_entry['plot-title'][:175])
    print(latest_entry['plot-title'])
    plt.savefig(f"trials-after-1.6/{i*10:05}.png", dpi=150, bbox_inches='tight')
    plt.close()

[I 2025-04-26 21:55:42,785] A new study created in memory with name: no-name-03abcba5-9a8c-4d14-af8e-bfc17f3dca82


Epoch 1/20


c:\Users\ic2594\AppData\Local\miniconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0036 - mae: 0.0426
Epoch 2/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0023 - mae: 0.0331
Epoch 3/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0023 - mae: 0.0336
Epoch 4/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0024 - mae: 0.0345
Epoch 5/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0026 - mae: 0.0350
Epoch 6/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0023 - mae: 0.0338
Epoch 7/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0025 - mae: 0.0340
Epoch 8/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0023 - mae: 0.0332
Epoch 9/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0024 - mae: 0.0332
Epoch 10/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0025 - mae: 0.0340
Epoch 11/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0026 - mae: 0.0345
Epoch 12/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0023 - mae: 0.0334
Epoch 13/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/s

[I 2025-04-26 21:56:12,896] Trial 0 finished with value: 0.00241635600104928 and parameters: {'lstm_units': 59, 'num_layers': 3, 'batch_size': 64, 'dropout_rate': 1, 'learning_rate': 0.01, 'lstm_activation': 'relu', 'features': 'all', 'episode_length': 39}. Best is trial 0 with value: 0.00241635600104928.


...

c:\Users\ic2594\AppData\Local\miniconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


77/77 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - loss: 0.0036 - mae: 0.0423
Epoch 2/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - loss: 0.0024 - mae: 0.0336
Epoch 3/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - loss: 0.0026 - mae: 0.0344
Epoch 4/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - loss: 0.0024 - mae: 0.0332
Epoch 5/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - loss: 0.0024 - mae: 0.0336
Epoch 6/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - loss: 0.0025 - mae: 0.0336
Epoch 7/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - loss: 0.0024 - mae: 0.0332
Epoch 8/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - loss: 0.0024 - mae: 0.0340
Epoch 9/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - loss: 0.0024 - mae: 0.0336
Epoch 10/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - loss: 0.0025 - mae: 0.0335
Epoch 11/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 0.0024 - mae: 0.0340
Epoch 12/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - loss: 0.0023 - mae: 0.0329
Epoch 13/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/s

[I 2025-04-26 22:34:56,691] Trial 27 finished with value: 0.0023916203062981367 and parameters: {'lstm_units': 75, 'num_layers': 4, 'batch_size': 64, 'dropout_rate': 1, 'learning_rate': 1e-05, 'lstm_activation': 'relu', 'features': 'nosentiment', 'episode_length': 54}. Best is trial 16 with value: 0.00239144591614604.


39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
154/154 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step
EL: 54; HP: {"lstm_units": 75, "num_layers": 4, "epochs": 20, "batch_size": 64, "dropout": 0.05, "learning_rate": 1e-05, "LSTM_activation": "relu", "__+ABD_exceptLast": false, "__finalAct": "sigmoid", "optimizer": "adam", "features": "nosentiment", "episode_length": 54, "layers": [75, 75, 75, 75], "loss": "mean_squared_error", "input_shape": [54, 8]}
Epoch 1/20


c:\Users\ic2594\AppData\Local\miniconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# Extract trial parameters and results
trials_data = [
    {**trial.params, 'score (MSE)': trial.value}
    for trial in study.trials
]

# Convert to a DataFrame
trials_df = pd.DataFrame(trials_data)

# Display the DataFrame
trials_df.sort_values(by="score (MSE)")

[(11593, 67, 10), (11593,), (2915, 67, 10), (2915,), (11593,), (2915,)]